# SETTING ENVIRONMENT


In [92]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [93]:
# set folder tempat kerja (current working directory)
import os
cwd = '/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita'
# cwd = '/content/drive/MyDrive/Monitoring Berita'

os.chdir(cwd)

# INPUT PARAMETER PENCARIAN

In [94]:
# AI Model (deepseek/openai)
model_name          = "openai"
# Model version (openai only: gpt-3.5-turbo/gpt-4o-mini/gpt-5-nano/gpt-5-mini/gpt-5)
model_version       = "gpt-5-nano"
# Tanggal awal pencarian berita
start_date          = "2025-09-29"
# Tanggal akhir pencarian berita
end_date            = "2025-09-29"
# Keywords pencarian berita
keywords_pencarian  = ["purbaya", "mbg", "koperasi merah putih"]
# Keywords analisis topic
keywords_topic      = ["bantuan sosial", "makan bergizi gratis", "tax amnesty"]
# Jumlah halaman pencarian
page_length         = 2

# LANGKAH ANALISIS

## SIMPAN PARAMETER PENCARIAN

In [95]:
# tandai waktu mulai proses
import time 
start_process_time = time.time()

In [96]:
from datetime import datetime, timedelta

# Konversi string ke datetime
start = datetime.strptime(start_date, "%Y-%m-%d")
end = datetime.strptime(end_date, "%Y-%m-%d")

# Buat list tanggal dari end_date ke start_date (descending)
search_date = [(end - timedelta(days=i)).strftime("%Y-%m-%d")
               for i in range((end - start).days + 1)]

In [97]:
import json
from pathlib import Path

config_path = Path("config.json")

# Data baru yang ingin disimpan/diupdate
variabel = {
    "search_date": search_date,
    "keywords": keywords_pencarian,
    "topic_keywords":keywords_topic,
    "max_page_length": page_length,
    "AI_name": model_name,
    "model_version": model_version
}

# Baca isi config.json lama (jika ada)
if config_path.exists():
    try:
        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)
    except Exception as e:
        print(f"Gagal membaca config.json: {e}")
        config = {}
else:
    config = {}

# Update hanya key yang ada di `variabel`
config.update(variabel)

# Simpan kembali ke file
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4, ensure_ascii=False)

print(f"Config.json berhasil diperbarui: {config_path}")


Config.json berhasil diperbarui: config.json


## SCRAP DATA BERITA DI MEDIA ONLINE

### detik.com

In [98]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_detik_search_method.ipynb')

%run "$file_loc"

In [99]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_detik.ipynb')

%run "$file_loc"

09:30:44 | INFO | Proses tanggal 2025-09-29 | Halaman 1 | URL: https://news.detik.com/berita/indeks?page=1&date=09/29/2025
09:30:45 | INFO |   Ditemukan 168 item artikel pada halaman ini.
09:30:45 | INFO |   Baru: 141 | Duplikat: 26 | Total unik 141.
09:30:46 | INFO | Proses tanggal 2025-09-29 | Halaman 2 | URL: https://news.detik.com/berita/indeks?page=2&date=09/29/2025
09:30:47 | INFO |   Ditemukan 168 item artikel pada halaman ini.
09:30:47 | INFO |   Baru: 0 | Duplikat: 167 | Total unik 141.
09:30:47 | INFO |   Tidak ada URL baru pada halaman ini (konten berulang). Stop iterasi tanggal ini.


,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Lalin di Terminal Kampung Rambutan Macet Imbas...,6 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8135129/lalin-...,False,
1,"Bareskrim Tangkap Kurir Narkoba di Jakut, 10 B...",16 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8135121/baresk...,False,
2,Cerita Jaksa di Bantaeng Alami Intimidasi Saat...,43 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8135073/cerita...,False,
3,Kementrans Akan Perluas Program Pengiriman Tra...,48 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8135087/kement...,False,
4,Legislator Setuju Koki Makan Bergizi Gratis Ha...,50 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8135085/legisl...,False,
...,...,...,...,...,...,...
136,Disclaimer,2025-09-29,Tidak Diketahui,https://www.detik.com/disclaimer,False,
137,Insertlive,2025-09-29,Tidak Diketahui,https://www.insertlive.com/,False,
138,Beautynesia,2025-09-29,Tidak Diketahui,https://beautynesia.id,False,
139,Female Daily,2025-09-29,Tidak Diketahui,https://femaledaily.com,False,



--- Ringkasan ---
Total artikel: 141
Artikel relevan: 8
5 contoh judul relevan:
1. Dapur-Chef Dievaluasi Buntut Menu MBG Bikin Siswa Keracunan
2. Juru Masak MBG Dievaluasi
3. JPPI Nilai Sertifikat Higienis dan CCTV Belum Bisa Selesaikan Persoalan MBG
4. Terungkap Bakteri 'Biang Kerok' Keracunan MBG di Bandung Barat
5. Anggota DPR Minta Penanganan Keracunan MBG Tak Cuma CCTV-Sertifikat Higienis


### cnbcindoncesia.com

In [100]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_cnbc.ipynb')

%run "$file_loc"

09:30:47 | INFO | Inisialisasi selesai.
09:30:47 | INFO | Memproses tanggal: 2025-09-29
09:30:47 | INFO | [2025-09-29] Page 1 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=1 | Artikel ditemukan (kandidat): 10
09:30:47 | INFO | [2025-09-29] Page 1 | Artikel cocok tanggal 2025-09-29: 10
09:30:49 | INFO | [2025-09-29] Page 2 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=2 | Artikel ditemukan (kandidat): 10
09:30:49 | INFO | [2025-09-29] Page 2 | Artikel cocok tanggal 2025-09-29: 10
09:30:50 | INFO | [2025-09-29] Page 3 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=3 | Artikel ditemukan (kandidat): 10
09:30:50 | INFO | [2025-09-29] Page 3 | Artikel cocok tanggal 2025-09-29: 10
09:30:51 | INFO | [2025-09-29] Page 4 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=4 | Artikel ditemukan (kandidat): 10
09:30:51 | INFO | [2025-09-29] Page 4 | Artikel cocok tanggal 2025-09-29: 10
09:30:52 | INFO | [2025-09-29] Page 5 | URL: https://www

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,"Asing Bawa Kabur Rp 42 T, Risiko Gagal Bayar R...",2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/research/2025092...,False,
1,"Daftar Bunga Deposito Rupiah & Dolar BRI, BNI-...",2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/research/2025092...,False,
2,Sepatu Impor Serang Pasar Senen dan Jatinegara...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250928200...,False,
3,Rekor Lagi! Harga Emas Antam Logam Mulia Hari ...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/research/2025092...,False,
4,Bank Syariah Nasional Resmi Dapat Izin Operasi...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509290...,False,
5,"IHSG Dibuka Menguat, Balik Lagi ke Level 8.100...",2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509290...,False,
6,Pemerintah Tetapkan Tarif Listrik Oktober-Dese...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250929075...,False,
7,"Breaking! Rupiah Perkasa, Dolar AS Turun ke Rp...",2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509290...,False,
8,InternasionalRaja Singa Tiba-Tiba Meledak di S...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250929084...,False,
9,Netizen Teriak M-banking BCA & blu Kompak Erro...,2025-09-29,Tidak Diketahui,https://www.cnbcindonesia.com/market/202509290...,False,



--- Ringkasan ---
Total artikel: 60
Artikel relevan: 6
Contoh judul relevan:
1. Penyebab Keracunan MBG Dibeberkan Profesor Eks Direktur WHOTech55 menit yang lalu
2. 4 Poin Penting Dibahas Purbaya & Bos BI Saat Santap Bebek GorengMarket1 jam yang lalu
3. Purbaya Bicara Soal Nasib BI di RUU P2SK, Hilang Independensi?Market1 jam yang lalu
4. Anggito Jadi Ketua LPS, Purbaya Usul ke Prabowo Tak Tambah Wamen LagiMarket2 jam yang lalu
5. Purbaya: Saya Koboi, Pistolnya Banyak Tapi Gak SembaranganMarket3 jam yang lalu


### idttimes.com

In [101]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_idn_times_search_method.ipynb')

%run "$file_loc"

### kompas.com

In [102]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_kompas.ipynb')

%run "$file_loc"

09:31:40 | INFO | Environment ready. Libraries loaded.
09:31:40 | INFO | Memproses tanggal: 2025-09-29, site: all
09:31:40 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-29&page=1
09:31:41 | INFO | Kandidat link artikel di page 1: 40
09:31:41 | INFO | Artikel berhasil diparse dari page 1: 39
09:31:42 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-29&page=2
09:31:42 | INFO | Kandidat link artikel di page 2: 41
09:31:42 | INFO | Artikel berhasil diparse dari page 2: 40
09:31:43 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-29&page=3
09:31:44 | INFO | Kandidat link artikel di page 3: 41
09:31:44 | INFO | Artikel berhasil diparse dari page 3: 40
09:31:45 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-29&page=4
09:31:45 | INFO | Kandidat link artikel di page 4: 41
09:31:45 | INFO | Artikel berhasil diparse dari page 4: 40
09:31:46 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-29&page=5
09:31:47 | INF

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,"Bon Appetit, Your Majesty Tamat dengan Rating ...",2025-09-29,Tidak Diketahui,https://www.kompas.com/hype/read/2025/09/29/09...,False,
1,"Gelar Media Briefing “No One Left Behind”, Kem...",2025-09-29,Tidak Diketahui,https://nasional.kompas.com/read/2025/09/29/09...,False,
2,Idul Fitri 2026 Tanggal Berapa? Ini Penetapan ...,2025-09-29,Tidak Diketahui,https://www.kompas.com/tren/read/2025/09/29/09...,False,
3,Perbaikan Jalan Nasional di Bima Telan APBN Rp...,2025-09-29,Tidak Diketahui,https://www.kompas.com/properti/read/2025/09/2...,False,
4,Qori Cilik Ditampar Pimpinan Pesantren di Palo...,2025-09-29,Tidak Diketahui,https://www.kompas.com/sulawesi-selatan/read/2...,False,
...,...,...,...,...,...,...
218,"Rapor Calvin Verdonk Usai Lawan Lyon, Catat Ra...",2025-09-29,Tidak Diketahui,https://bola.kompas.com/read/2025/09/29/004338...,False,
219,"Hasil Newcastle United Vs Arsenal 1-2, Dua Gol...",2025-09-29,Tidak Diketahui,https://bola.kompas.com/read/2025/09/29/004223...,False,
220,"Cara Upload File ke Google Drive Melalui Link,...",2025-09-29,Tidak Diketahui,https://tekno.kompas.com/read/2025/09/29/00350...,False,
221,"Hasil Lille Vs Lyon 0-1, Calvin Verdonk Masih ...",2025-09-29,Tidak Diketahui,https://bola.kompas.com/read/2025/09/29/001421...,False,



--- Ringkasan ---
Total artikel: 223
Artikel relevan: 12
Contoh 5 judul relevan:
1. Belum Ada Laporan Kasus Keracunan MBG, Pemkot Bekasi Tetap Evaluasi SPPGNEWS29/09/2025
2. Usai KLB MBG, Pemerintah Siapkan Ahli Gizi hingga Perbaiki Tata KelolaNEWS29/09/2025
3. Disdik Kalteng Ajak Warga Awasi Program MBG lewat Aplikasi, Bagaimana Caranya?REGIONAL29/09/2025
4. Kartu Pers Istana Jurnalis CNN Dicabut Usai Tanya soal MBG, Mensesneg Tegaskan Prabowo Tak TerlibatPROV29/09/2025
5. Soal Program MBG, Zulhas: SPPG yang Bermasalah DitutupPROV29/09/2025


### liputan6.com

In [103]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_liputan6.ipynb')

%run "$file_loc"

09:31:49 | INFO | Memproses tanggal [News]: 2025-09-29
09:31:49 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=1
09:31:50 | INFO | News halaman 1: ditemukan 18 artikel
09:31:51 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=2
09:31:51 | INFO | News halaman 2: ditemukan 4 artikel
09:31:52 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=3
09:31:53 | INFO | News halaman 3: ditemukan 4 artikel
09:31:54 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=4
09:31:54 | INFO | News halaman 4: ditemukan 4 artikel
09:31:55 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=5
09:31:55 | INFO | News halaman 5: ditemukan 4 artikel
09:31:56 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/29?page=6
09:31:57 | INFO | News halaman 6: ditemukan 4 artikel
09:31:58 | INFO | Memproses tanggal [Bisnis]: 2025-09-29
09:31:58 | INFO | GET Bisnis: https://www.liputan6.com/bisnis/ind

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Deretan Hoaks yang Dikaitkan dengan BBM Pertal...,2025-09-28T18:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/617064...,False,
1,Kumpulan Artikel Palsu yang Dikaitkan dengan J...,2025-09-28T16:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/617060...,False,
2,Ragam Artikel Hoaks Terbaru yang Sempat Viral ...,2025-09-28T14:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/617058...,False,
3,"Deretan Video Hoaks Terbaru yang Beredar, Jang...",2025-09-28T11:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/617056...,False,
4,Sejumlah DPW-DPC PPP Tak Terima Diklaim Hadir ...,2025-09-29T08:43:12+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6171049/sej...,False,
5,Metro Sepekan: Istri Gus Dur hingga Mantan Men...,2025-09-29T08:30:08+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6170656/met...,False,
6,Cara Mudah Cek Online Bansos Rp 600 Ribu Pakai...,2025-09-29T08:10:07+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6171038/car...,False,
7,Top 3 News: Anggota TNI Pelaku Pemukulan Karya...,2025-09-29T08:00:10+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6170877/top...,False,
8,"Unggah Foto Bareng Donald Trump, Prabowo Yakin...",2025-09-29T07:49:22+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6171054/ung...,False,
9,"Oktober 2025, Simak Daftar Hari Besar Nasional...",2025-09-29T07:29:41+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6171010/okt...,False,



==== RINGKASAN ====
Total artikel: 23
Artikel relevan: 1
Contoh 5 judul relevan:
1. Prabowo Rapat Bareng Menteri di Kertanegara Minggu Malam, Bahas MBG hingga Migas


### cnnindonesia.com

In [104]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_cnn.ipynb')

%run "$file_loc"

09:32:05 | INFO | Project root: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita
09:32:05 | INFO | Keywords: ['purbaya', 'mbg', 'koperasi merah putih']
09:32:05 | INFO | Using TAG mode for: purbaya
09:32:11 | INFO | [tag:purbaya] page 1 -> 36 items
09:32:19 | INFO | [tag:purbaya] page 2 -> 34 items
09:32:26 | INFO | [tag:purbaya] page 3 -> 28 items
09:32:27 | INFO | Using TAG mode for: mbg
09:32:31 | INFO | [tag:mbg] page 1 -> 34 items
09:32:38 | INFO | [tag:mbg] page 2 -> 34 items
09:32:45 | INFO | [tag:mbg] page 3 -> 36 items
09:32:45 | INFO | Using TAG mode for: koperasi merah putih
09:32:54 | INFO | [tag:koperasi merah putih] page 1 -> 36 items
09:33:02 | INFO | [tag:koperasi merah putih] page 2 -> 36 items
09:33:10 | INFO | [tag:koperasi merah putih] page 3 -> 36 items
09:33:12 | INFO | Total unique links: 118
09:33:12 | INFO | Saved: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05.

,judul_berita,url_berita,keyword,page,published_at,scraped_at
0,Keluar,https://connect.detik.com/oauth/signout?redire...,purbaya,1,NaT,2025-09-29 09:32:06.552070+07:00
1,DAFTAR,https://connect.detik.com/accounts/register?cl...,purbaya,1,NaT,2025-09-29 09:32:06.598476+07:00
2,Politik,https://www.cnnindonesia.com/nasional/politik,purbaya,1,NaT,2025-09-29 09:32:06.734374+07:00
3,Hukum & Kriminal,https://www.cnnindonesia.com/nasional/hukum-kr...,purbaya,1,NaT,2025-09-29 09:32:06.807830+07:00
4,Peristiwa,https://www.cnnindonesia.com/nasional/peristiwa,purbaya,1,NaT,2025-09-29 09:32:06.969751+07:00
5,Pemilu,https://www.cnnindonesia.com/nasional/pemilu,purbaya,1,NaT,2025-09-29 09:32:07.094142+07:00
6,Info Politik,https://www.cnnindonesia.com/nasional/info-pol...,purbaya,1,NaT,2025-09-29 09:32:07.232659+07:00
7,Asia Pasifik,https://www.cnnindonesia.com/internasional/asi...,purbaya,1,NaT,2025-09-29 09:32:07.439193+07:00
8,Timur Tengah,https://www.cnnindonesia.com/internasional/tim...,purbaya,1,NaT,2025-09-29 09:32:07.548930+07:00
9,Eropa Amerika,https://www.cnnindonesia.com/internasional/ero...,purbaya,1,NaT,2025-09-29 09:32:07.682984+07:00


Output Excel: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/daftar_berita/cnn_links.csv


### okezone.com

In [105]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_okezone.ipynb')

%run "$file_loc"

09:33:12 | WARNING | Config tidak ditemukan di /content/drive/MyDrive/Monitoring Berita/config.json. Menggunakan default.
09:33:12 | INFO | topic_keywords: ['ekonomi', 'bursa', 'saham', 'investasi', 'keuangan', 'bank', 'rupiah', 'inflasi', 'bisnis']
09:33:12 | INFO | dates: ['2025-09-29']
09:33:12 | INFO | max_pages_per_date: 5
09:33:12 | INFO | Memproses tanggal: 2025-09-29
09:33:14 | INFO | [2025-09-29] URL: https://index.okezone.com/bydate/index/2025/09/29/0/ | Artikel ditemukan: 15
09:33:14 | INFO | Artikel diparsing (page 1): 15
09:33:25 | WARNING | Timeout https://index.okezone.com/home/index/15/ (try 1/3)
09:33:32 | INFO | [2025-09-29] URL: https://index.okezone.com/home/index/15/ | Artikel ditemukan: 15
09:33:32 | INFO | Artikel diparsing (page 2): 15
09:33:42 | INFO | [2025-09-29] URL: https://index.okezone.com/home/index/30/ | Artikel ditemukan: 15
09:33:42 | INFO | Artikel diparsing (page 3): 15
09:33:53 | WARNING | Timeout https://index.okezone.com/home/index/45/ (try 1/3)


,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,"IHSG Dibuka Menguat ke 8.139, Saham-Saham Ini ...","Market Update | Senin, 29 September 2025 09:21...",Tidak Diketahui,https://economy.okezone.com/read/2025/09/29/27...,True,saham
1,Penyebab Kapten Timnas Indonesia Jay Idzes Tol...,"Liga Italia | Senin, 29 September 2025 09:13 W...",Tidak Diketahui,https://bola.okezone.com/read/2025/09/29/47/31...,False,
2,Shell Buka Suara soal Isu PHK Massal Imbas Keb...,"Hot Issue | Senin, 29 September 2025 09:10 WIB...",Tidak Diketahui,https://economy.okezone.com/read/2025/09/29/32...,False,
3,Kebakaran Hebat di Taman Sari akibat Kipas Angin,"Megapolitan | Senin, 29 September 2025 09:04 W...",Tidak Diketahui,https://news.okezone.com/read/2025/09/29/338/3...,False,
4,"BCA Mobile Berangsur Normal Usai Error, Ini In...","Hot Issue | Senin, 29 September 2025 08:58 WIB...",Tidak Diketahui,https://economy.okezone.com/read/2025/09/29/32...,False,
...,...,...,...,...,...,...
104,Redaksi,2025-09-29,Tidak Diketahui,https://management.okezone.com/redaksi,False,
105,Kotak Pos,2025-09-29,Tidak Diketahui,https://management.okezone.com/pos,False,
106,Karier,2025-09-29,Tidak Diketahui,https://career.okezone.com,False,
107,Info Iklan,2025-09-29,Tidak Diketahui,https://client.okezone.com/advertising,False,



--- Ringkasan ---
Total artikel: 109
Artikel relevan: 2

Contoh 5 judul relevan:
- IHSG Dibuka Menguat ke 8.139, Saham-Saham Ini Cuan  |  keywords: saham  |  https://economy.okezone.com/read/2025/09/29/278/3173154/ihsg-dibuka-menguat-ke-8-139-saham-saham-ini-cuan
- Bedu Bongkar Alasan Ceraikan Istri, Bantah karena Masalah Ekonomi  |  keywords: ekonomi  |  https://celebrity.okezone.com/read/2025/09/29/33/3173133/bedu-bongkar-alasan-ceraikan-istri-bantah-karena-masalah-ekonomi


## MENGGABUNGKAN DATA

### Konsolidasi Data Url Berita

In [106]:
file_loc = os.path.join(cwd, '01_konsolidasi_data_berita.ipynb')

%run "$file_loc"

Membaca Excel:   0%|          | 0/9 [00:00<?, ?file/s]

09:34:05 | INFO | Duplikat dihapus: 25 baris | Total akhir: 605
09:34:05 | INFO | Selesai gabung. Total baris: 605 | File terbaca: 9
09:34:05 | WARNING | Gagal parse index=359, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=367, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=368, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=369, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=370, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=371, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=372, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=373, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | WARNING | Gagal parse index=375, nilai='<NA>': String does not contain a date: <NA>
09:34:05 | W

File berhasil disimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/konsolidasi_berita/hasil_gabungan_20250929_093405.csv


## BACA BERITA

### Membaca Data Url Berita

In [107]:
file_loc

'/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/01_konsolidasi_data_berita.ipynb'

In [108]:
file_loc = os.path.join(cwd, '02_baca_link_berita.ipynb')

%run "$file_loc"

09:34:07 | INFO | Total URL untuk diproses: 326 (limit=None)


Scraping artikel:   0%|          | 0/326 [00:00<?, ?it/s]

09:37:36 | INFO | 
Summary status:
status
ok_trafilatura      207
ok_site_specific    119
Name: count, dtype: int64
09:37:36 | INFO | Timeout sample rows: 0
09:37:36 | INFO | Tersimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/hasil_baca_berita/hasil_scraping_artikel_20250929_093736.csv
09:37:36 | INFO | Berhasil update config.json di /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/config.json


## LABELLING ISU


In [109]:
file_loc = os.path.join(cwd, '03_labelling_awal.ipynb')

%run "$file_loc"

Normalisasi tanggal_berita: total=326, sukses=298, gagal=28 (8.6%)
Filter tanggal: 2025-09-29 s.d 2025-09-29 (dari 298 -> 112)
Labeling selesai. Jumlah baris: 112


,kategori_isu,jumlah
0,Isu Lainnya,41
1,Isu Nasional,33
2,Isu Internasional,25
3,Isu Kemenkeu,13


Tersimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/hasil_labeling/hasil_labeling_20250929_093738.csv & /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/hasil_labeling/hasil_labeling_20250929_093738.xlsx
Berhasil update config.json di /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/monitoring-berita/config.json


## ANALISIS DENGAN GEN AI

In [110]:
file_loc = os.path.join(cwd, '04_analisis_dengan_genAI.ipynb')

%run "$file_loc"

API keys - Deepseek: OK | OpenAI: OK
Provider yang dipakai: openai


Analisis berita (parallel openai):   0%|          | 0/71 [00:00<?, ?berita/s]

09:37:41 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:42 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:42 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:43 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:45 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:46 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:47 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:47 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:47 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:49 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:37:50 | INFO | HT

Pipeline selesai. Hasil disimpan di analisis_ai_20250929_openai_default.csv
Monitoring sementara: {'execution_datetime': '2025-09-29 09:38:51', 'provider': 'openai', 'model_name': 'gpt-5-nano', 'elapsed_seconds': 72.36, 'output_file': 'analisis_ai_20250929_openai_default.csv', 'n_rows_result_df': 71}
Monitoring log ditulis/ditambahkan ke monitoring_log_genai.csv
    execution_datetime provider  model_name  elapsed_seconds  \
0  2025-09-29 09:38:51   openai  gpt-5-nano            72.36   

                               output_file  n_rows_in_memory  \
0  analisis_ai_20250929_openai_default.csv                71   

   n_rows_output_file  
0                  71  


09:38:55 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Hasil uji LLM (openai): {'topik': 'Kemenkeu', 'subtopik': 'Lainnya', 'sentimen': 'netral', 'alasan_sentimen': 'Berita ini hanya menyampaikan informasi tentang kebijakan tanpa memberikan penilaian positif atau negatif.', 'poin_of_interest': 'Menteri Keuangan', 'statement_pejabat': 'Menteri Keuangan menyampaikan kebijakan fiskal terbaru terkait pajak dan APBN.', 'confidence': 0.9}
Gagal membaca monitoring log: Error tokenizing data. C error: Expected 7 fields in line 10, saw 13



# REKAP

In [111]:
# Tandai waktu selesai proses
end_process_time = time.time()
elapsed_time = end_process_time - start_process_time
# jadikan menit & detik
elapsed_minutes = int(elapsed_time // 60)
elapsed_seconds = int(elapsed_time % 60)
print(f"Waktu proses: {elapsed_minutes} menit {elapsed_seconds} detik")

Waktu proses: 9 menit 10 detik
